In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf

# For inline plots in Jupyter
%matplotlib inline
import pandas as pd
import matplotlib as plt
import glob
import os
import xml.etree.ElementTree as ET
from datetime import datetime, timezone
from tqdm import tqdm
import codecs
import csv
import openpyxl

In [2]:
data_directory_weather = "/home/paulharford/college/project/project_data/processed/WEATHERED_warnings_2014-2023_cleaned_v2.csv"
data_directory_ihfd = "/home/paulharford/college/project/project_data/processed/ihfd_clean.csv"
data_directory_census = "/home/paulharford/college/project/project_data/processed/census_estimated_per_region_2014_2023.csv"
full_path_ihfd = os.path.abspath(data_directory_ihfd)
full_path_weather = os.path.abspath(data_directory_weather)
full_path_census = os.path.abspath(data_directory_census)

In [3]:
df_ihfd = pd.read_csv(full_path_ihfd)
df_weather = pd.read_csv(full_path_weather)
df_census = pd.read_csv(full_path_census)

In [4]:
df_ihfd.head()

,New Health Regions,NOCA_TraumaPeriodDay,Adm_First_Pres_Hosp_DateTime,NOCA_FirstPresPeriodDay,NOCA_AgeRange,NOCA_Gender,LOS,Adm_Trauma_TYPE,Adm_Ward_Type,Adm_Pre_Frac_Indoor,...,Adm_Ger_Acute_Assess,Adm_Operation,Adm_Asa_Grade,Adm_Anaesthesia,Adm_Surgery_Delay_Reason,Adm_Mobilised,Adm_Pressure_Ulcers,Adm_Spec_Falls_Assess,Adm_Bone_Protect_Med,Adm_Multi_Rehab_Assess
0,HSE Mid West,AM,2016-09-06 00:00:00.000000000,PM,70-74,Female,12,2.0,1.0,2.0,...,1.0,8.0,3.0,5.0,7.0,1.0,2.0,1.0,1.0,1.0
1,HSE Mid West,AM,2016-09-15 00:00:00.000000000,Night,90-94,Female,4,2.0,1.0,2.0,...,1.0,1.0,2.0,5.0,0.0,1.0,2.0,1.0,1.0,1.0
2,HSE Mid West,AM,2016-09-14 00:00:00.000000000,AM,75-79,Male,6,2.0,1.0,2.0,...,1.0,8.0,2.0,5.0,0.0,1.0,2.0,1.0,5.0,1.0
3,HSE Mid West,AM,2016-09-04 00:00:00.000000000,AM,90-94,Female,16,2.0,1.0,2.0,...,2.0,1.0,3.0,5.0,0.0,1.0,2.0,1.0,1.0,1.0
4,HSE Mid West,AM,2016-08-23 00:00:00.000000000,Night,65-69,Male,29,2.0,1.0,2.0,...,1.0,8.0,3.0,5.0,2.0,1.0,2.0,1.0,1.0,1.0


In [5]:
df_ihfd.rename(columns={"New Health Regions": "Region"}, inplace=True)

In [6]:
df_ihfd.rename(columns={"NOCA_AgeRange": "Age_Range"}, inplace=True)

In [7]:
df_ihfd["Adm_First_Pres_Hosp_DateTime"] = pd.to_datetime(df_ihfd["Adm_First_Pres_Hosp_DateTime"])
df_ihfd["date"] = df_ihfd["Adm_First_Pres_Hosp_DateTime"].dt.date
df_ihfd['date'] = pd.to_datetime(df_ihfd['date'])
df_ihfd['Year'] = df_ihfd['date'].dt.year

In [8]:
hip_agg = (
    df_ihfd
    .groupby(['Region', 'date', 'Year','Age_Range', 'NOCA_Gender', 'LOS'])
    .agg(Fractures=('date', 'count'))
    .reset_index()
)

In [9]:
hip_agg.head(10)

,Region,date,Year,Age_Range,NOCA_Gender,LOS,Fractures
0,HSE Dublin and Midlands,2013-01-10,2013.0,85-89,Female,6,1
1,HSE Dublin and Midlands,2013-01-11,2013.0,85-89,Male,14,1
2,HSE Dublin and Midlands,2013-01-11,2013.0,90-94,Male,26,1
3,HSE Dublin and Midlands,2013-01-15,2013.0,60-64,Female,20,1
4,HSE Dublin and Midlands,2013-01-17,2013.0,85-89,Female,7,1
5,HSE Dublin and Midlands,2013-01-18,2013.0,75-79,Female,20,1
6,HSE Dublin and Midlands,2013-01-20,2013.0,85-89,Female,23,1
7,HSE Dublin and Midlands,2013-01-22,2013.0,85-89,Female,35,1
8,HSE Dublin and Midlands,2013-01-25,2013.0,65-69,Female,22,1
9,HSE Dublin and Midlands,2013-01-25,2013.0,80-84,Male,5,1


In [10]:
import pandas as pd
import plotly.express as px
from shiny import App, ui, reactive, render
from shiny import App, ui, reactive
from shinywidgets import output_widget, render_widget  #

In [11]:
# 1) Define a user interface
app_ui = ui.page_fluid(
    ui.h2("Hip Fracture Incidents in Ireland"),
    ui.layout_sidebar(
        ui.sidebar(
            ui.input_select(
                "region_select",
                "Choose a region:",
                choices=sorted(hip_agg["Region"].unique()),
                selected=sorted(hip_agg["Region"].unique())[0]
            ),
            ui.input_slider(
                "year_range",
                "Select Year Range:",
                min=hip_agg["Year"].min(),
                max=hip_agg["Year"].max(),
                value=(hip_agg["Year"].min(), hip_agg["Year"].max()),
                step=1
            ),
            # Add new filters for Age_Range
            ui.input_checkbox_group(
                "age_range_select",
                "Select Age Ranges:",
                choices=sorted(hip_agg["Age_Range"].unique()),
                selected=sorted(hip_agg["Age_Range"].unique())
            ),
            # Add filter for Gender
            ui.input_checkbox_group(
                "gender_select",
                "Select Gender:",
                choices=sorted(hip_agg["NOCA_Gender"].unique()),
                selected=sorted(hip_agg["NOCA_Gender"].unique())
            ),
            # Add a visualization type selector
            ui.input_radio_buttons(
                "viz_type",
                "Visualization Type:",
                choices=["Total Counts", "By Age Range", "By Gender"],
                selected="Total Counts"
            )
        ),
        output_widget("fractures_plot")
    )
)

# 2) Define the server setup
def server(input, output, session):
    @reactive.Calc
    def filtered_data():
        selected_region = input.region_select()
        year_min, year_max = input.year_range()
        selected_age_ranges = input.age_range_select()
        selected_genders = input.gender_select()
        
        # Filter by all selected criteria
        dff = hip_agg[
            (hip_agg["Region"] == selected_region) &
            (hip_agg["Year"] >= year_min) &
            (hip_agg["Year"] <= year_max) &
            (hip_agg["Age_Range"].isin(selected_age_ranges)) &
            (hip_agg["NOCA_Gender"].isin(selected_genders))
        ]
        
        return dff
    
    @render_widget
    def fractures_plot():
        dff = filtered_data()
        viz_type = input.viz_type()
        
        if dff.empty:
            # Return a blank figure or indicate no data
            fig = px.line(title="No data available for selection.")
            return fig
        
        if viz_type == "Total Counts":
            # Count fractures by year
            fracture_counts = dff.groupby("Year").size().reset_index(name="Fractures")
            
            fig = px.line(
                fracture_counts,
                x="Year",
                y="Fractures",
                title=f"Hip Fractures in {input.region_select()}",
                markers=True
            )
            
        elif viz_type == "By Age Range":
            # Group by Year and Age_Range
            age_counts = dff.groupby(["Year", "Age_Range"]).size().reset_index(name="Fractures")
            
            fig = px.line(
                age_counts,
                x="Year",
                y="Fractures",
                color="Age_Range",
                title=f"Hip Fractures by Age Range in {input.region_select()}",
                markers=True
            )
            
        elif viz_type == "By Gender":
            # Group by Year and Gender
            gender_counts = dff.groupby(["Year", "NOCA_Gender"]).size().reset_index(name="Fractures")
            
            fig = px.line(
                gender_counts,
                x="Year",
                y="Fractures",
                color="NOCA_Gender",
                title=f"Hip Fractures by Gender in {input.region_select()}",
                markers=True
            )
        
        # Improve the figure layout
        fig.update_layout(
            xaxis_title="Year",
            yaxis_title="Number of Fractures",
            legend_title_text="",
            template="plotly_white"
        )
            
        return fig

# 3) Create App object
app = App(app_ui, server)

In [12]:
import shiny
print(shiny.__version__)

1.2.1


In [13]:
import nest_asyncio
nest_asyncio.apply()

# Now you can run the app
app.run()


INFO:     Started server process [650730]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:40386 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:40386 - "GET /lib/requirejs-2.3.6/require.min.js HTTP/1.1" 200 OK
INFO:     127.0.0.1:40404 - "GET /lib/shiny-1.2.1/shiny.js HTTP/1.1" 200 OK
INFO:     127.0.0.1:40392 - "GET /lib/jquery-3.6.0/jquery-3.6.0.min.js HTTP/1.1" 200 OK
INFO:     127.0.0.1:40414 - "GET /lib/shiny-busy-indicators-1.2.1/busy-indicators.css HTTP/1.1" 200 OK
INFO:     127.0.0.1:40442 - "GET /lib/bootstrap-5.3.1/bootstrap.bundle.min.js HTTP/1.1" 200 OK
INFO:     127.0.0.1:40428 - "GET /lib/bootstrap-5.3.1/bootstrap.min.css HTTP/1.1" 200 OK
INFO:     127.0.0.1:40386 - "GET /lib/bslib-components-0.8.0.9000/web-components.min.js HTTP/1.1" 200 OK
INFO:     127.0.0.1:40414 - "GET /lib/bslib-components-0.8.0.9000/components.min.js HTTP/1.1" 200 OK
INFO:     127.0.0.1:40392 - "GET /lib/ionrangeslider-2.3.1/js/ion.rangeSlider.min.js HTTP/1.1" 200 OK
INFO:     127.0.0.1:40442 - "GET /lib/strftime-0.9.2/strftime-min.js HTTP/1.1" 200 OK
INFO:     127.0.0

INFO:     ('127.0.0.1', 40446) - "WebSocket /websocket/" [accepted]
INFO:     connection open


INFO:     127.0.0.1:40442 - "GET /lib/anywidget-0.0/index.js HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     connection closed
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [650730]
